<p style="font-size:32px;text-align:center"> <b>Social network Graph Link Prediction - Facebook Challenge</b> </p>

### Problem statement: 
Given a undirected social graph, have to predict missing links to recommend users (Link Prediction in graph)

### Mapping the problem into supervised learning problem:
- Generated training samples of good and bad links from given directed graph and for each link got some features like no of followers, is he followed back, page rank, katz score, adar index, some svd fetures of adj matrix, some weight features etc. and trained ml model based on these features to predict link. 
- Some reference papers and videos :  
    - https://www.cs.cornell.edu/home/kleinber/link-pred.pdf
    - https://www3.nd.edu/~dial/publications/lichtenwalter2010new.pdf
    - https://kaggle2.blob.core.windows.net/forum-message-attachments/2594/supervised_link_prediction.pdf
    - https://www.youtube.com/watch?v=2M77Hgy17cg

### Business objectives and constraints:  
- No low-latency requirement.
- Probability of prediction is useful to recommend highest probability links

### Performance metric for supervised learning:  
- Both precision and recall is important so F1 score is good choice
- Confusion matrix

# 在map feature 時,盡量不使用dic,節省記憶體空間之版本(可在所有eikoset上執行)

In [9]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")
import community
import csv
import pandas as pd#pandas to create small DataFrames 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
import os.path
import shutil
# to install xgboost: pip3 install xgboost
import xgboost as xgb
import warnings
import networkx as nx
import pdb
from argparse import ArgumentParser, Namespace
import random
random.seed(25)



In [10]:
def parse_args() -> Namespace:
    parser = ArgumentParser()
    parser.add_argument('--PCA', help='Whether to do PCA.',action='store_true')
    parser.add_argument('--eikoset_path', help='Path of the eikoset to be used.',default='./gemsec_facebook_eikoset/facebook_clean_eiko/tvshow_edges.csv')

    args = parser.parse_args(args=['--PCA',\
                                  '--eikoset_path','./gemsec_facebook_dataset/facebook_clean_data/tvshow_edges.csv'])
 
    
    #     args = parser.parse_args()
    
    return args


args = parse_args()    

if args.PCA:
    PCAornot='PCA'
else:
    PCAornot='noPCA'
    
base_name=os.path.basename(args.eikoset_path)

genre=base_name.split('.')[0].split('_')
genre.pop()
genre="_".join(genre)

print(args.PCA)
print(args.eikoset_path)

True
./gemsec_facebook_dataset/facebook_clean_data/tvshow_edges.csv


In [11]:
# 移除eiko
shutil.rmtree('eiko')

In [12]:
# 建立eiko資料夾
os.makedirs(f'./eiko/{genre}/after_eda', exist_ok=True)
os.makedirs(f'./eiko/{genre}/fea_sample/other_features', exist_ok=True)


In [13]:
#建立results資料夾
os.makedirs(f"./results/{PCAornot}/{genre}/RandomForest", exist_ok=True)
os.makedirs(f"./results/{PCAornot}/{genre}/GBDT", exist_ok=True)

In [14]:
#reading graph
if not os.path.isfile(f'./eiko/{genre}/after_eda/train_woheader.csv'):
    traincsv = pd.read_csv(args.eikoset_path)
    print(traincsv[traincsv.isna().any(1)])
    print(traincsv.info())
    print("Number of duplicate entries: ",sum(traincsv.duplicated()))
    print("saved the graph into file")
    traincsv.to_csv(f'./eiko/{genre}/after_eda/train_woheader.csv',header=False,index=False)
    g=nx.read_edgelist(f'./eiko/{genre}/after_eda/train_woheader.csv',delimiter=',',create_using=nx.Graph(),nodetype=int)
        
    print(nx.info(g))
else:
    g=nx.read_edgelist(f'./eiko/{genre}/after_eda/train_woheader.csv',delimiter=',',create_using=nx.Graph(),nodetype=int)
    print(nx.info(g))

Empty DataFrame
Columns: [node_1, node_2]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17262 entries, 0 to 17261
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   node_1  17262 non-null  int64
 1   node_2  17262 non-null  int64
dtypes: int64(2)
memory usage: 269.8 KB
None
Number of duplicate entries:  0
saved the graph into file
Graph with 3892 nodes and 17262 edges


In [15]:
with open('/media/paul/G/social-network-final-project/GEMSEC/GEMSEC_nodes.pickle', 'rb') as f:
    GEMSEC_nodes = pickle.load(f)

GEMSEC_nodes==list(g.nodes)

True

In [16]:
GEMSEC_nodes

[0,
 1838,
 1744,
 14,
 2543,
 1,
 1009,
 1171,
 1465,
 2069,
 2080,
 1856,
 3799,
 1033,
 2717,
 300,
 1603,
 942,
 3678,
 952,
 815,
 2,
 3656,
 3459,
 1157,
 1993,
 3,
 386,
 2659,
 2160,
 1925,
 3511,
 372,
 1506,
 2303,
 1634,
 1706,
 3038,
 749,
 2790,
 452,
 1936,
 2915,
 2972,
 1100,
 2918,
 3026,
 2406,
 1673,
 11,
 3156,
 2250,
 2350,
 1050,
 1060,
 1457,
 3054,
 72,
 1553,
 1114,
 3525,
 2064,
 3791,
 1595,
 1177,
 1596,
 2935,
 545,
 1840,
 423,
 1788,
 2890,
 603,
 1498,
 703,
 1073,
 1422,
 3283,
 2590,
 3431,
 354,
 1720,
 3291,
 2773,
 2804,
 2476,
 2036,
 46,
 1986,
 2588,
 3507,
 566,
 4,
 667,
 1931,
 5,
 275,
 1874,
 3832,
 2847,
 1929,
 3237,
 1209,
 578,
 3852,
 1452,
 2361,
 590,
 2005,
 77,
 3409,
 2816,
 80,
 2454,
 1119,
 771,
 3204,
 2201,
 2764,
 1609,
 315,
 3550,
 1193,
 2344,
 1370,
 2593,
 2472,
 3221,
 3062,
 3747,
 3082,
 2713,
 911,
 6,
 556,
 2524,
 3839,
 2355,
 7,
 2170,
 495,
 3189,
 2109,
 1920,
 1961,
 3651,
 2179,
 3339,
 938,
 48,
 2549,
 3122

In [7]:
# print(sorted(list(g.nodes)))

> Displaying a sub graph

In [8]:
# if not os.path.isfile('train_woheader_sample.csv'):
#     pd.read_csv('./gemsec_facebook_eikoset/facebook_clean_eiko/tvshow_edges.csv', nrows=50).to_csv('train_woheader_sample.csv',header=False,index=False)
    
# subgraph=nx.read_edgelist('train_woheader_sample.csv',delimiter=',',create_using=nx.Graph(),nodetype=int)
# # https://stackoverflow.com/questions/9402255/drawing-a-huge-graph-with-networkx-and-matplotlib

# pos=nx.spring_layout(subgraph)
# nx.draw(subgraph,pos,node_color='#A0CBE2',edge_color='#00bb5e',width=1,edge_cmap=plt.cm.Blues,with_labels=True)
# plt.savefig("graph_sample.pdf")
# print(nx.info(subgraph))

# 1. Exploratory eiko Analysis

In [9]:
# # No of Unique persons 
# print("The number of unique persons",len(g.nodes()))

## 1.1 No of neighbors for each person

In [10]:
# degree_dist = list(dict(g.degree()).values())

# degree_dist.sort()

# plt.figure(figsize=(10,6))
# plt.plot(degree_dist)
# plt.xlabel('Index No')
# plt.ylabel('No Of neighbors')
# plt.show()

In [11]:
# plt.boxplot(degree_dist)
# plt.ylabel('No Of neighbors')
# plt.show()

In [12]:
# ### 90-100 percentile
# for i in range(0,11):
#     print(90+i,'percentile value is',np.percentile(degree_dist,90+i))

99% of eiko having followers of 40 only.

In [13]:
# ### 99-100 percentile
# for i in range(10,110,10):
#     print(99+(i/100),'percentile value is',np.percentile(degree_dist,99+(i/100)))

In [14]:
# %matplotlib inline
# sns.set_style('ticks')
# fig, ax = plt.subplots()
# fig.set_size_inches(11.7, 8.27)
# sns.distplot(degree_dist, color='#16A085')
# plt.xlabel('PDF of degree')
# sns.despine()
# #plt.show()

In [15]:
# print('No of persons having neighbor less than 10 are',np.sum(np.array(degree_dist)<10))

In [16]:
# print('No of connected components',len(list(nx.connected_components(g))))
# count=0
# for i in list(nx.connected_components(g)):
#     if len(i)==2:
#         count+=1
# print('connected components with 2 nodes',count)

# 2. Posing a problem as classification problem 

## 2.1 Generating some edges which are not present in graph for supervised learning  
Generated Bad links from graph which are not in graph and whose shortest path is greater than 2. 

In [17]:
# ##generating bad edges from given graph

# if not os.path.isfile(f'./eiko/{genre}/after_eda/missing_edges_final.p'):
#     #getting all set of edges
#     r = csv.reader(open(f'./eiko/{genre}/after_eda/train_woheader.csv','r'))
#     edges = dict()
#     for edge in r:
#         edges[(int(edge[0]), int(edge[1]))] = 1
        
#     missing_edges = set([])
#     g_edges_num=len(g.edges)
#     g_nodes_num=len(g.nodes)
#     while (len(missing_edges)<g_edges_num):
#         a=random.randint(0,g_nodes_num-1)
#         b=random.randint(0,g_nodes_num-1)
#         tmp = edges.get((a,b),-1)
#         if tmp == -1 and a!=b:
#             try:
#                 if nx.shortest_path_length(g,source=a,target=b) > 2: 

#                     missing_edges.add((a,b))
#                 else:
#                     continue  
#             except:  
#                     missing_edges.add((a,b))
#             missing_edges.add((a,b))
#         else:
#             continue
# #         print(len(missing_edges))
#     pickle.dump(missing_edges,open(f'./eiko/{genre}/after_eda/missing_edges_final.p','wb'))
# else:
#     missing_edges = pickle.load(open(f'./eiko/{genre}/after_eda/missing_edges_final.p','rb'))

In [18]:
# #getting all set of edges
# r = csv.reader(open(f'./eiko/{genre}/after_eda/train_woheader.csv','r'))
# edges = dict()
# for edge in r:
#     edges[(int(edge[0]), int(edge[1]))] = 1
#     edges[(int(edge[1]), int(edge[0]))] = 1
# missing_edges = set([])
# g_edges_num=len(g.edges)
# g_nodes_num=len(g.nodes)
# while (len(missing_edges)<g_edges_num):
#     a=random.randint(0,g_nodes_num-1)
#     b=random.randint(0,g_nodes_num-1)
# #     print((a,b))
#     tmp = edges.get((a,b),-1)
#     if tmp == -1 and a!=b:

#         missing_edges.add((a,b))
#     else:
#         continue




In [19]:
#保雲
adj_G = nx.to_numpy_matrix(g,nodelist=sorted(list(g.nodes)))
missing_edges = []
g_edges_num=len(g.edges)
g_nodes_num=len(g.nodes)
cnt = 0
# unconnected_link = set([])
while cnt < g_edges_num:
    i = random.randint(0, g_nodes_num-1)
    j = random.randint(0, g_nodes_num-1)
    if adj_G[i, j] == 0 and i!=j:
        adj_G[i, j] = 1
        missing_edges.append([i,j])
        cnt += 1
      

In [20]:
# with open('code/node_list.pickle', 'rb') as f:
#     node_list = pickle.load(f)
# print(node_list)

In [21]:
with open('code/unconnected_link.pickle', 'rb') as f:
    unconnected_link = pickle.load(f)
len(unconnected_link)
# print(len(sorted(unconnected_link)))

17262

## 2.2 Training and Test eiko split:  
Removed edges from Graph and used as test eiko and after removing used that graph for creating features for Train and test eiko

In [22]:
from sklearn.model_selection import train_test_split
if (not os.path.isfile(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv')) and (not os.path.isfile(f'./eiko/{genre}/after_eda/test_pos_after_eda.csv')):
    #reading total eiko df
    df_pos = pd.read_csv(args.eikoset_path)
    df_neg = pd.DataFrame(missing_edges, columns=['source_node', 'destination_node'])
    
    print("Number of nodes in the graph with edges", df_pos.shape[0])
    print("Number of nodes in the graph without edges", df_neg.shape[0])
    
    #Trian test split 
    #Spiltted eiko into 80-20 
    #positive links and negative links seperatly because we need positive training eiko only for creating graph 
    #and for feature generation
    X_train_pos, X_test_pos, y_train_pos, y_test_pos  = train_test_split(df_pos,np.ones(len(df_pos)),test_size=0.2, random_state=9)
    X_train_neg, X_test_neg, y_train_neg, y_test_neg  = train_test_split(df_neg,np.zeros(len(df_neg)),test_size=0.2, random_state=9)
    
    print('='*60)
    print("Number of nodes in the train eiko graph with edges", X_train_pos.shape[0],"=",y_train_pos.shape[0])
    print("Number of nodes in the train eiko graph without edges", X_train_neg.shape[0],"=", y_train_neg.shape[0])
    print('='*60)
    print("Number of nodes in the test eiko graph with edges", X_test_pos.shape[0],"=",y_test_pos.shape[0])
    print("Number of nodes in the test eiko graph without edges", X_test_neg.shape[0],"=",y_test_neg.shape[0])

    #removing header and saving
    X_train_pos.to_csv(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv',header=False, index=False)
    X_test_pos.to_csv(f'./eiko/{genre}/after_eda/test_pos_after_eda.csv',header=False, index=False)
    X_train_neg.to_csv(f'./eiko/{genre}/after_eda/train_neg_after_eda.csv',header=False, index=False)
    X_test_neg.to_csv(f'./eiko/{genre}/after_eda/test_neg_after_eda.csv',header=False, index=False)
del missing_edges

Number of nodes in the graph with edges 17262
Number of nodes in the graph without edges 17262
Number of nodes in the train eiko graph with edges 13809 = 13809
Number of nodes in the train eiko graph without edges 13809 = 13809
Number of nodes in the test eiko graph with edges 3453 = 3453
Number of nodes in the test eiko graph without edges 3453 = 3453


In [23]:
if (os.path.isfile(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv')) and (os.path.isfile(f'./eiko/{genre}/after_eda/test_pos_after_eda.csv')):        
    train_graph=nx.read_edgelist(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.Graph(),nodetype=int)
    test_graph=nx.read_edgelist(f'./eiko/{genre}/after_eda/test_pos_after_eda.csv',delimiter=',',create_using=nx.Graph(),nodetype=int)
    print(nx.info(train_graph))
    print(nx.info(test_graph))

    # finding the unique nodes in the both train and test graphs
    train_nodes_pos = set(train_graph.nodes())
    test_nodes_pos = set(test_graph.nodes())

    trY_teY = len(train_nodes_pos.intersection(test_nodes_pos))
    trY_teN = len(train_nodes_pos - test_nodes_pos)
    teY_trN = len(test_nodes_pos - train_nodes_pos)

    print('no of people common in train and test -- ',trY_teY)
    print('no of people present in train but not present in test -- ',trY_teN)

    print('no of people present in test but not present in train -- ',teY_trN)
    print(' % of people not there in Train but exist in Test in total Test eiko are {} %'.format(teY_trN/len(test_nodes_pos)*100))

Graph with 3754 nodes and 13809 edges
Graph with 2406 nodes and 3453 edges
no of people common in train and test --  2268
no of people present in train but not present in test --  1486
no of people present in test but not present in train --  138
 % of people not there in Train but exist in Test in total Test eiko are 5.7356608478802995 %


> we have a cold start problem here

In [24]:
#final train and test eiko sets
if (not os.path.isfile(f'./eiko/{genre}/after_eda/train_after_eda.csv')) and \
(not os.path.isfile(f'./eiko/{genre}/after_eda/test_after_eda.csv')) and \
(not os.path.isfile(f'./eiko/{genre}/train_y.csv')) and \
(not os.path.isfile(f'./eiko/{genre}/test_y.csv')) and \
(os.path.isfile(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv')) and \
(os.path.isfile(f'./eiko/{genre}/after_eda/test_pos_after_eda.csv')) and \
(os.path.isfile(f'./eiko/{genre}/after_eda/train_neg_after_eda.csv')) and \
(os.path.isfile(f'./eiko/{genre}/after_eda/test_neg_after_eda.csv')):
    
    X_train_pos = pd.read_csv(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv', names=['source_node', 'destination_node'])
    X_test_pos = pd.read_csv(f'./eiko/{genre}/after_eda/test_pos_after_eda.csv', names=['source_node', 'destination_node'])
    X_train_neg = pd.read_csv(f'./eiko/{genre}/after_eda/train_neg_after_eda.csv', names=['source_node', 'destination_node'])
    X_test_neg = pd.read_csv(f'./eiko/{genre}/after_eda/test_neg_after_eda.csv', names=['source_node', 'destination_node'])

    print('='*60)
    print("Number of nodes in the train eiko graph with edges", X_train_pos.shape[0])
    print("Number of nodes in the train eiko graph without edges", X_train_neg.shape[0])
    print('='*60)
    print("Number of nodes in the test eiko graph with edges", X_test_pos.shape[0])
    print("Number of nodes in the test eiko graph without edges", X_test_neg.shape[0])

    X_train = X_train_pos.append(X_train_neg,ignore_index=True)
    y_train = np.concatenate((y_train_pos,y_train_neg))
    X_test = X_test_pos.append(X_test_neg,ignore_index=True)
    y_test = np.concatenate((y_test_pos,y_test_neg)) 
    
    X_train.to_csv(f'./eiko/{genre}/after_eda/train_after_eda.csv',header=False,index=False)
    X_test.to_csv(f'./eiko/{genre}/after_eda/test_after_eda.csv',header=False,index=False)
    pd.DataFrame(y_train.astype(int)).to_csv(f'./eiko/{genre}/train_y.csv',header=False,index=False)
    pd.DataFrame(y_test.astype(int)).to_csv(f'./eiko/{genre}/test_y.csv',header=False,index=False)
    
    print("eiko points in train eiko",X_train.shape)
    print("eiko points in test eiko",X_test.shape)
    print("Shape of traget variable in train",y_train.shape)
    print("Shape of traget variable in test", y_test.shape)

Number of nodes in the train eiko graph with edges 13809
Number of nodes in the train eiko graph without edges 13809
Number of nodes in the test eiko graph with edges 3453
Number of nodes in the test eiko graph without edges 3453
eiko points in train eiko (27618, 2)
eiko points in test eiko (6906, 2)
Shape of traget variable in train (27618,)
Shape of traget variable in test (6906,)


In [25]:
with open('code/pos_xtrain.pickle', 'rb') as f:
    pos_xtrain = pickle.load(f)

In [26]:
pos_xtrain[['node_1','node_2']].values==X_train_pos.values

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       ...,
       [ True,  True],
       [ True,  True],
       [ True,  True]])

In [27]:
pos_xtrain 


,node_1,node_2,link
17062,3413,3343,1
1548,146,2234,1
11120,1481,1113,1
13872,2036,2350,1
759,65,2031,1
...,...,...,...
4532,455,2848,1
4673,478,3275,1
5014,529,1038,1
9979,1214,2751,1


In [28]:
X_train_pos

,source_node,destination_node
0,3413,3343
1,146,2234
2,1481,1113
3,2036,2350
4,65,2031
...,...,...
13804,455,2848
13805,478,3275
13806,529,1038
13807,1214,2751


**Computed and store the eiko for featurization**

In [29]:
with open('code/xtrain.pickle', 'rb') as f:
    xtrain1 = pickle.load(f)

In [30]:
xtrain1.reset_index(inplace=True, drop=True)

In [31]:
X_train[['source_node','destination_node']].iloc[13809]

source_node         2971
destination_node    1522
Name: 13809, dtype: int64

In [32]:
xtrain1[['node_1','node_2']].iloc[13809]

node_1    2971
node_2    1522
Name: 13809, dtype: int64

In [33]:
cp=X_train[['source_node','destination_node']].values==xtrain1[['node_1','node_2']].values


not_equal_index=[]
for i,c in enumerate(cp):
    if c[0]==False or c[1]==False:
        print('not equal')
        not_equal_index+=[i]
print(not_equal_index)

[]


In [34]:
with open('code/xtest.pickle', 'rb') as f:
    xtest1 = pickle.load(f)

# 3. Features definition

In [35]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small DataFrames 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm

In [36]:
g_nodes_num

3892

In [37]:
X_train_pos = pd.read_csv(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv', names=['source_node', 'destination_node'])

In [38]:
X_train_pos

,source_node,destination_node
0,3413,3343
1,146,2234
2,1481,1113
3,2036,2350
4,65,2031
...,...,...
13804,455,2848
13805,478,3275
13806,529,1038
13807,1214,2751


In [39]:
def SavedfLink(df):
    node1 = df['node_1'].tolist()
    node2 = df['node_2'].tolist()
    link = []
    for i in range(len(node1)):
        link.append((node1[i], node2[i]))
    print("# of Link:", len(link))
    return link

In [40]:
def SavedfLink1(df):
    node1 = df['source_node'].tolist()
    node2 = df['destination_node'].tolist()
    link = []
    for i in range(len(node1)):
        link.append((node1[i], node2[i]))
    print("# of Link:", len(link))
    return link

In [41]:
pos_train_link = SavedfLink(pos_xtrain)
X_train_pos_link = SavedfLink1(X_train_pos)


# of Link: 13809
# of Link: 13809


In [42]:
sorted(pos_train_link)==sorted(X_train_pos_link)

True

In [43]:
if (0,14) in X_train_pos_link:
    print('yes')

yes


In [44]:
X_train_pos = pd.read_csv(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv', names=['source_node', 'destination_node'])
X_train_pos

if (0,14) in X_train_pos.values:
    print('yes')

yes


In [5]:
if os.path.isfile(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv'):
    t=nx.read_edgelist(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.Graph(),nodetype=int)

In [6]:
t.edges([0])

EdgeDataView([(0, 1838), (0, 14), (0, 1744), (0, 2543)])

In [8]:
if (14,0) in t.edges:
    print('yes')
# train_graph.edges

for p in t.edges:
    if p==(14,0):
        print('yes1')

yes
yes1


In [48]:
if (0,14) in list(train_graph.edges):
    print('yes')
# list(train_graph./edges)

# 3.1 Reading eiko

In [49]:
if os.path.isfile(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist(f'./eiko/{genre}/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.Graph(),nodetype=int)
    train_graph.add_nodes_from(range(g_nodes_num))
    #     print(nx.info(train_graph))
else:
    print("please run the FB_EDA.ipynb or download the files from drive")

In [50]:
len(train_graph.nodes)

3892

In [51]:
with open('code/train_graph.pickle', 'rb') as f:
    train_graph1 = pickle.load(f)


# 3.2. Similarity measures

## 3.2.1 Jaccard distance:
http://www.statisticshowto.com/jaccard-index/

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

In [52]:
# #To do: Jaccard distance(yl)
# #input:one node pair
# #output:feature of that node pair
# def jaccard_distance(a,b):
#     if train_graph.has_node(a) and train_graph.has_node(b):
#         try:
#             return next( nx.jaccard_coefficient(train_graph, [(a,b)]) )[2]
#         except:
#             return 0
#     else:
#         return 0

In [53]:
def jaccard_distance(a,b):
    return next( nx.jaccard_coefficient(train_graph, [(a,b)]) )[2]



In [54]:
next( nx.jaccard_coefficient(train_graph, [(1882,0)]) )[2]

0.0

In [55]:
if 1882 in train_graph.nodes:
    print('yes')

yes


## 3.2.2 Cosine distance

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

In [56]:
# def cosine_distance(a,b):
#     if train_graph.has_node(a) and train_graph.has_node(b):
#         X = set( train_graph.neighbors(a))
#         Y = set( train_graph.neighbors(b))
#         return len( X&Y ) / ( len(X)*len(Y) )
#     return 0

In [57]:
def cosine_distance(a,b):
    X = set( train_graph.neighbors(a))

    Y = set( train_graph.neighbors(b))

    try:
        return len( X&Y ) / ( len(X)*len(Y) )
    except:
        return 0


# 3.3 Other Graph Features

## 3.3.1 Shortest path:
Getting Shortest path between twoo nodes, if nodes have direct path i.e directly connected then we are removing that edge and calculating path. 

In [58]:
# #To do: Shortest path(yl)
# #input:one node pair
# #output:feature of that node pair
# def compute_shortest_path_length(a,b):
#     if train_graph.has_node(a) and train_graph.has_node(b):
#         try:
#             return nx.shortest_path_length( train_graph, source=a, target=b)
#         except:
#             return 0   
#     else:
#         return 0

In [59]:
def compute_shortest_path_length(a,b):
    try:
        return nx.shortest_path_length( train_graph, source=a, target=b)
    except:
        return 0   


In [60]:
# for i,j in df_final_train[['source_node', 'destination_node']].values:
#     print(nx.shortest_path_length(train_graph, source=i, target=j))

## 3.3.2 Checking for same community

In [61]:
# #getting connected edges from graph 
# cc_dic={}
# for i,c in enumerate(nx.connected_components(train_graph)):
#     for node in c:
#         cc_dic[node]=i

In [62]:
# def belongs_to_same_cc(a,b):
#     if cc_dic.get(a,-1)==cc_dic.get(b,-1) and cc_dic.get(a,-1)!=-1:
#         return 1
#     else:
#         return 0

In [63]:
# partition = community.best_partition(train_graph)
# def same_community(a, b):
#     if partition.get(a) and partition.get(b):
#         return int(partition[a] == partition[b])
#     else:
#         return 0

In [64]:
partition = community.best_partition(train_graph)
def same_community(a, b): 
    return int(partition[a] == partition[b])

## 3.3.3 Common neighbors

In [65]:
# #To do: Common neighbors(yl)
# def common_neighbors(a,b):
#     if train_graph.has_node(a) and train_graph.has_node(b):
#         return len(sorted(nx.common_neighbors(train_graph,a,b)))
#     return 0

In [66]:
def common_neighbors(a,b):
    return len(sorted(nx.common_neighbors(train_graph,a,b)))

## 3.3.4 Adamic/Adar Index:
Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

In [67]:
# #To do: adar(yl)
# #input:one node pair
# #output:feature of that node pair
# def calc_adar(a,b):
#     if train_graph.has_node(a) and train_graph.has_node(b):
#         try:
#             return next( nx.adamic_adar_index(train_graph, [(a,b)]) )[2]
#         except:
#             return 0
#     else:
#         return 0

In [68]:
#To do: adar(yl)
#input:one node pair
#output:feature of that node pair
def calc_adar(a,b):
    try:
        return next( nx.adamic_adar_index(train_graph, [(a,b)]) )[2]
    except:
        return 0


## 3.3.5 Resource Allocation Index:
Among a number of similarity-based methods to predict missing links in a complex network, Research Allocation Index performs well with lower time complexity. It is defined as a fraction of a resource that a node can send to another through their common neighbors.

$$
\text { Research Allocation Index }(\mathrm{X}, \mathrm{Y})=\Sigma_{u \in N(X) \cap N(Y)} 1 /|N(u)|
$$

In [69]:
# def Resource_Allocation_Index(a,b):
#     if train_graph.has_node(a) and train_graph.has_node(b):
#         try:
#             return next( nx.resource_allocation_index(train_graph, [(a,b)]) )[2]
#         except:
#             return 0
#     else:
#         return 0


In [70]:
def Resource_Allocation_Index(a,b):
    try:
        return next( nx.resource_allocation_index(train_graph, [(a,b)]) )[2]
    except:
        return 0


## 3.3.6 Preferential Attachment:
Preferential attachment means that the more connected a node is, the more likely it is to receive new links (refer to this article for referring to Barabasi Albert graph formed on the concepts of Preferential Attachment) Nodes with higher degree gets more neighbors.
$$
\text { Preferential Attachment }(X, Y)=|N(X)| \cdot|N(Y)|
$$

In [71]:
# def Preferential_Attachment(a,b):
#     if train_graph.has_node(a) and train_graph.has_node(b):
#         try:
#             return next( nx.preferential_attachment(train_graph, [(a,b)]) )[2]
#         except:
#             return 0
#     else:
#         return 0


In [72]:
def Preferential_Attachment(a,b):
    try:
        return next( nx.preferential_attachment(train_graph, [(a,b)]) )[2]
    except:
        return 0



## 3.3.7 SVD

In [73]:
#for svd features to get feature vector creating a dict node val and inedx in svd vector
sadj_col = sorted(train_graph.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

In [74]:
Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()

In [75]:
U, s, V = svds(Adj, k = 6)
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (3892, 3892)
U Shape (3892, 6)
V Shape (6, 3892)
s Shape (6,)


In [76]:
def svd(x, S):
    try:
        z = sadj_dict[x]
        return S[z]
    except:
        return [0,0,0,0,0,0]

## 3.3.8 SVD dot

Add feature called svd_dot. you can calculate svd_dot as Dot product between sourse node svd and destination node svd features. you can read about this in below pdf https://storage.googleapis.com/kaggle-forum-message-attachments/2594/supervised_link_prediction.pdfm

In [77]:
def svd_dot(S,D):
    list_var = []
    A = S.values
    B = D.values
    for i in range(len(A)):
        list_var.append(np.dot(A[i],B[i]))
    df = pd.DataFrame(list_var)
    return df

## 3.3.9 GEMSEC

In [78]:
#To do: GEMSEC
#input:one node pair
#output:feature of that node pair
def GEMSEC_embedding(a,b):
    a_embedding=GEMSEC.iloc[g_nodes.index(a), 0]
    b_embedding=GEMSEC.iloc[g_nodes.index(b), 0]
    return a_embedding+b_embedding 
    

## 3.3.10 GEMSECWithRegularization

In [79]:
def GEMSECWithRegularization_embedding(a,b):
    a_embedding=GEMSECWithRegularization.iloc[g_nodes.index(a), 0]
    b_embedding=GEMSECWithRegularization.iloc[g_nodes.index(b), 0]
    return a_embedding+b_embedding
        

## 3.3.11 DeepWalk

In [80]:
def DeepWalk_embedding(a,b):
    a_embedding=DeepWalk.iloc[g_nodes.index(a), 0]
    b_embedding=DeepWalk.iloc[g_nodes.index(b), 0]
    return a_embedding+b_embedding
        

## 3.3.12 DeepWalkWithRegularization

In [81]:
def DeepWalkWithRegularization_embedding(a,b):
    a_embedding=DeepWalkWithRegularization.iloc[g_nodes.index(a), 0]
    b_embedding=DeepWalkWithRegularization.iloc[g_nodes.index(b), 0]
    return a_embedding+b_embedding

# 4. Featurization

## 4.1 Reading eiko from both train and test

In [82]:
df_final_train = pd.read_csv(f'./eiko/{genre}/after_eda/train_after_eda.csv', names=['source_node', 'destination_node'])
df_final_train['indicator_link'] = pd.read_csv(f'./eiko/{genre}/train_y.csv', names=['indicator_link'])

print("Our train matrix size ",df_final_train.shape)
df_final_train.head(2)

Our train matrix size  (27618, 3)


,source_node,destination_node,indicator_link
0,3413,3343,1
1,146,2234,1


In [83]:
df_final_test = pd.read_csv(f'./eiko/{genre}/after_eda/test_after_eda.csv', names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv(f'./eiko/{genre}/test_y.csv', names=['indicator_link'])


print("Our test matrix size ",df_final_test.shape)
df_final_test.head(2)

Our test matrix size  (6906, 3)


,source_node,destination_node,indicator_link
0,191,538,1
1,1464,2816,1


## 4.2 Adding a set of features(one dimension feature)

__we will create these each of these features for both train and test eiko points__
<ol>
<li>Jaccard Distance</li>
<li>Cosine Distance</li>
<li>Shortest Path</li>
<li>Same Community</li>
<li>Common Neighbors</li>
<li>Adamic/Adar Index</li>
<li>Resource Allocation Index</li>
<li>Preferential Attachment</li>
</ol>

In [84]:
if not os.path.isfile(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage1.h5'):
    #mapping Jaccard Distance to train and test eiko
    df_final_train['jaccard_distance'] = df_final_train.apply(lambda row:
                                            jaccard_distance(row['source_node'],row['destination_node']),axis=1)
    df_final_test['jaccard_distance'] = df_final_test.apply(lambda row:
                                            jaccard_distance(row['source_node'],row['destination_node']),axis=1)

    #mapping Cosine Distance to train and test eiko
    df_final_train['cosine_distance'] = df_final_train.apply(lambda row:
                                            cosine_distance(row['source_node'],row['destination_node']),axis=1)
    df_final_test['cosine_distance'] = df_final_test.apply(lambda row:
                                            cosine_distance(row['source_node'],row['destination_node']),axis=1)
    
    #mapping Checking for same community to train and test eiko
    df_final_train['same_community'] = df_final_train.apply(lambda row:
                                            same_community(row['source_node'],row['destination_node']),axis=1)
    df_final_test['same_community'] = df_final_test.apply(lambda row:
                                            same_community(row['source_node'],row['destination_node']),axis=1)
    
    
    
    #mapping Shortest path to train and test eiko
    df_final_train['shortest_path'] = df_final_train.apply(lambda row:
                                            compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)
    df_final_test['shortest_path'] = df_final_test.apply(lambda row:
                                            compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)

    #mapping Common neighbors to train and test eiko
    df_final_train['Common_neighbors'] = df_final_train.apply(lambda row:
                                            common_neighbors(row['source_node'],row['destination_node']),axis=1)
    df_final_test['Common_neighbors'] = df_final_test.apply(lambda row:
                                            common_neighbors(row['source_node'],row['destination_node']),axis=1)
    
    #mapping Adamic/Adar Index to train and test eiko
    df_final_train['adar'] = df_final_train.apply(lambda row:
                                            calc_adar(row['source_node'],row['destination_node']),axis=1)
    df_final_test['adar'] = df_final_test.apply(lambda row:
                                            calc_adar(row['source_node'],row['destination_node']),axis=1)

    #mapping Resource Allocation Index to train and test eiko
    df_final_train['Resource_Allocation_Index'] = df_final_train.apply(lambda row:
                                            Resource_Allocation_Index(row['source_node'],row['destination_node']),axis=1)
    df_final_test['Resource_Allocation_Index'] = df_final_test.apply(lambda row:
                                            Resource_Allocation_Index(row['source_node'],row['destination_node']),axis=1)

    #mapping Preferential Attachment to train and test eiko
    df_final_train['Preferential_Attachment'] = df_final_train.apply(lambda row:
                                            Preferential_Attachment(row['source_node'],row['destination_node']),axis=1)
    df_final_test['Preferential_Attachment'] = df_final_test.apply(lambda row:
                                            Preferential_Attachment(row['source_node'],row['destination_node']),axis=1)
    

    hdf = HDFStore(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage1.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()
else:
    df_final_train = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage1.h5', 'train_df',mode='r')
    df_final_test = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage1.h5', 'test_df',mode='r')   
    
    

In [85]:
# del cc_dic

In [86]:
del partition

## 4.3 Adding new set of features


#### 1.SVD features for both source and destination


In [87]:
if not os.path.isfile(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stag2.h5'):
    #===================================================================================================
    
    df_final_train[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
    df_final_train.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
    df_final_train[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
    df_final_train.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    #===================================================================================================
    
    df_final_train[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6']] = \
    df_final_train.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

    df_final_train[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
    df_final_train.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
    #===================================================================================================
    
    df_final_test[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
    df_final_test.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
    df_final_test[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
    df_final_test.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)

    #===================================================================================================
    
    df_final_test[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6']] = \
    df_final_test.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

    df_final_test[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
    df_final_test.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
    #===================================================================================================

    hdf = HDFStore(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage2.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close()
    
else:
    df_final_train = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage2.h5', 'train_df',mode='r')
    df_final_test = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage2.h5', 'test_df',mode='r')

In [88]:
del sadj_col
del sadj_dict
del Adj
del U
del s
del V

#### 2.SVD dot

In [89]:
if not os.path.isfile(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage3.h5'):
    u_s = df_final_train[['svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3', 'svd_u_s_4','svd_u_s_5', 'svd_u_s_6']]
    u_d = df_final_train[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4','svd_u_d_5', 'svd_u_d_6']]
    df_final_train['svd_dot_u'] = svd_dot(u_s,u_d)

    v_s = df_final_train[['svd_v_s_1', 'svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4','svd_v_s_5', 'svd_v_s_6']]
    v_d = df_final_train[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4','svd_v_d_5', 'svd_v_d_6']]
    df_final_train['svd_dot_v'] = svd_dot(v_s,v_d)

    u_s = df_final_test[['svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3', 'svd_u_s_4','svd_u_s_5', 'svd_u_s_6']]
    u_d = df_final_test[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4','svd_u_d_5', 'svd_u_d_6']]
    df_final_test['svd_dot_u'] = svd_dot(u_s,u_d)

    v_s = df_final_test[['svd_v_s_1', 'svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4','svd_v_s_5', 'svd_v_s_6']]
    v_d = df_final_test[['svd_v_s_1', 'svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4','svd_v_s_5', 'svd_v_s_6']]
    df_final_test['svd_dot_v'] = svd_dot(v_s,v_d)
    
    hdf = HDFStore(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage3.h5')
    hdf.put('train_df',df_final_train, format='table', data_columns=True)
    hdf.put('test_df',df_final_test, format='table', data_columns=True)
    hdf.close() 
else:
    df_final_train = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage3.h5', 'train_df',mode='r')
    df_final_test = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage3.h5', 'test_df',mode='r')

## 4.4 Adding new set of features(multiple dimension feature)

__we will create these each of these features for both train and test eiko points__
<ol>
<li>Node to vector</li>    
<li>GEMSEC</li>
<li>GEMSECWithRegularization</li>
<li>DeepWalk</li>
<li>DeepWalkWithRegularization</li>
</ol>

In [90]:
if args.PCA:
    if not os.path.isfile(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage4.h5'):
        #mapping Node to vector to train and test eiko
        from node2vec import Node2Vec
        from sklearn.decomposition  import PCA
        node2vec = Node2Vec(g, dimensions=100, walk_length=16, num_walks=50)
        n2v_model = node2vec.fit(window=7, min_count=1)
        embedding=[] 
        for n1,n2 in zip(df_final_train['source_node'],df_final_train['destination_node']):
            x=n2v_model.wv[str(n1)]+n2v_model.wv[str(n2)]
            embedding.append(x)    
        pca = PCA( n_components=1 )
        df_final_train['node2vector'] = pca.fit_transform(embedding)
 
        embedding=[]
        for n1,n2 in zip(df_final_test['source_node'],df_final_test['destination_node']):
            x=n2v_model.wv[str(n1)]+n2v_model.wv[str(n2)]
            embedding.append(x)    
        pca = PCA( n_components=1 )
        df_final_test['node2vector'] = pca.fit_transform(embedding)
        

        g_nodes=list(g.nodes)
        GEMSEC=pd.read_csv(f"./GEMSEC/output_PCA/GEMSEC/PCA/{genre}_embedding.csv")
        GEMSECWithRegularization=pd.read_csv(f"./GEMSEC/output_PCA/GEMSECWithRegularization/PCA/{genre}_embedding.csv")
        DeepWalk=pd.read_csv(f"./GEMSEC/output_PCA/DeepWalk/PCA/{genre}_embedding.csv")
        DeepWalkWithRegularization=pd.read_csv(f"./GEMSEC/output_PCA/DeepWalkWithRegularization/PCA/{genre}_embedding.csv")
    
        #mapping GEMSEC to train and test eiko
        df_final_train['GEMSEC'] = df_final_train.apply(lambda row:
                                                GEMSEC_embedding(row['source_node'],row['destination_node']),axis=1)
        df_final_test['GEMSEC'] = df_final_test.apply(lambda row:
                                                GEMSEC_embedding(row['source_node'],row['destination_node']),axis=1)    

        #mapping GEMSECWithRegularization to train and test eiko
        df_final_train['GEMSECWithRegularization'] = df_final_train.apply(lambda row:
                                                GEMSECWithRegularization_embedding(row['source_node'],row['destination_node']),axis=1)
        df_final_test['GEMSECWithRegularization'] = df_final_test.apply(lambda row:
                                                GEMSECWithRegularization_embedding(row['source_node'],row['destination_node']),axis=1)

        #mapping DeepWalk to train and test eiko
        df_final_train['DeepWalk'] = df_final_train.apply(lambda row:
                                                DeepWalk_embedding(row['source_node'],row['destination_node']),axis=1)
        df_final_test['DeepWalk'] = df_final_test.apply(lambda row:
                                                DeepWalk_embedding(row['source_node'],row['destination_node']),axis=1)    

        #mapping DeepWalkWithRegularization to train and test eiko
        df_final_train['DeepWalkWithRegularization'] = df_final_train.apply(lambda row:
                                                DeepWalkWithRegularization_embedding(row['source_node'],row['destination_node']),axis=1)
        df_final_test['DeepWalkWithRegularization'] = df_final_test.apply(lambda row:
                                                DeepWalkWithRegularization_embedding(row['source_node'],row['destination_node']),axis=1)          

        hdf = HDFStore(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage4.h5')
        hdf.put('train_df',df_final_train, format='table', data_columns=True)
        hdf.put('test_df',df_final_test, format='table', data_columns=True)
        hdf.close()
        
        
        del node2vec
        del n2v_model
        del embedding
        del pca
        del g_nodes
        del GEMSEC
        del GEMSECWithRegularizationjaccard_coefficient
        del DeepWalk
        del DeepWalkWithRegularization
    else:
        df_final_train = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage4.h5', 'train_df',mode='r')
        df_final_test = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage4.h5', 'test_df',mode='r')   
 

Computing transition probabilities:   0%|          | 0/3892 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|████████████████| 50/50 [00:09<00:00,  5.23it/s]


**Store the eiko as final for machine learning models**

# RandomForest

In [111]:
#reading
from pandas import read_hdf
if args.PCA:
    df_final_train = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage4.h5', 'train_df',mode='r')
    df_final_test = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage4.h5', 'test_df',mode='r')
else:
    df_final_train = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage3.h5', 'train_df',mode='r')
    df_final_test = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage3.h5', 'test_df',mode='r')    

In [92]:
df_final_train.columns

Index(['source_node', 'destination_node', 'indicator_link', 'jaccard_distance',
       'cosine_distance', 'same_community', 'shortest_path',
       'Common_neighbors', 'adar', 'Resource_Allocation_Index',
       'Preferential_Attachment', 'svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3',
       'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6', 'svd_u_d_1', 'svd_u_d_2',
       'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5', 'svd_u_d_6', 'svd_v_s_1',
       'svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',
       'svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5',
       'svd_v_d_6', 'svd_dot_u', 'svd_dot_v', 'node2vector', 'GEMSEC',
       'GEMSECWithRegularization', 'DeepWalk', 'DeepWalkWithRegularization'],
      dtype='object')

In [93]:
y_train = df_final_train.indicator_link
y_test = df_final_test.indicator_link

In [94]:
df_final_train.drop(['source_node', 'destination_node','indicator_link','svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6','svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6','svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6','svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6','shortest_path'],axis=1,inplace=True)
df_final_test.drop(['source_node', 'destination_node','indicator_link','svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6','svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6','svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6','svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6','shortest_path'],axis=1,inplace=True)
# df_final_train=df_final_train[['jaccard_distance','cosine_distance','same_community','Common_neighbors']]
# df_final_test=df_final_test[['jaccard_distance','cosine_distance','same_community','Common_neighbors']]

In [95]:
# df_final_train=df_final_train[['adar']]
# df_final_test=df_final_test[['adar']]

In [96]:
# df_final_train=df_final_train[['Resource_Allocation_Index']]
# df_final_test=df_final_test[['Resource_Allocation_Index']]

In [97]:
# df_final_train=df_final_train[['Preferential_Attachment']]
# df_final_test=df_final_test[['Preferential_Attachment']]

In [98]:
# df_final_train=df_final_train[['node2vector']]
# df_final_test=df_final_test[['node2vector']]

In [99]:
# df_final_train=df_final_train[['GEMSEC']]
# df_final_test=df_final_test[['GEMSEC']]

In [100]:
# df_final_train=df_final_train[['GEMSECWithRegularization']]
# df_final_test=df_final_test[['GEMSECWithRegularization']]

In [101]:
# df_final_train=df_final_train[['DeepWalk']]
# df_final_test=df_final_test[['DeepWalk']]

In [102]:
# df_final_train=df_final_train[['DeepWalkWithRegularization']]
# df_final_test=df_final_test[['DeepWalkWithRegularization']]

In [112]:
df_final_train

,source_node,destination_node,indicator_link,jaccard_distance,cosine_distance,same_community,shortest_path,Common_neighbors,adar,Resource_Allocation_Index,...,svd_v_d_4,svd_v_d_5,svd_v_d_6,svd_dot_u,svd_dot_v,node2vector,GEMSEC,GEMSECWithRegularization,DeepWalk,DeepWalkWithRegularization
0,3413,3343,1,0.294118,0.047619,1,1,5,2.161036,0.501354,...,-9.834833e-04,2.151501e-04,3.207119e-05,8.625683e-06,8.625683e-06,-0.844273,-2.185937,4.049660,2.901259,1.313043
1,146,2234,1,0.000000,0.000000,1,1,0,0.000000,0.000000,...,-2.180966e-06,5.250280e-09,1.054386e-08,2.157490e-10,2.157490e-10,2.375643,-3.882927,-7.951956,-11.163434,5.721247
2,1481,1113,1,0.176471,0.031250,1,1,3,1.392175,0.348485,...,-3.601277e-05,1.821458e-07,1.118372e-06,3.087866e-08,3.087866e-08,3.695516,-6.778196,-5.363354,-6.472065,3.890312
3,2036,2350,1,0.505618,0.010302,1,1,45,11.265735,0.839549,...,6.149393e-02,-7.842732e-04,1.308478e-01,1.262273e-02,1.262273e-02,-3.234747,9.163351,-2.573558,5.838528,0.805989
4,65,2031,1,0.529412,0.053571,1,1,9,3.733518,0.817671,...,-7.730326e-04,1.511492e-04,2.750821e-05,2.179123e-05,2.179123e-05,-1.218323,-4.380023,-6.899915,-6.302849,2.947937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27613,1787,3286,0,0.000000,0.000000,0,9,0,0.000000,0.000000,...,-1.004038e-09,3.579229e-12,4.689558e-13,9.583323e-12,9.583323e-12,1.161157,0.257331,5.421441,1.114054,-1.991831
27614,3373,2483,0,0.000000,0.000000,0,6,0,0.000000,0.000000,...,-1.636402e-01,2.227113e-04,3.582600e-02,7.461572e-08,7.461572e-08,0.405508,-6.737229,-6.573252,-8.216017,6.988257
27615,1496,549,0,0.000000,0.000000,0,5,0,0.000000,0.000000,...,-2.530197e-05,1.098556e-06,3.076463e-07,4.725180e-08,4.725180e-08,-0.040633,-5.601143,2.794851,-0.921274,-0.824722
27616,1484,2784,0,0.000000,0.000000,0,9,0,0.000000,0.000000,...,-9.646349e-07,6.801768e-08,3.836097e-09,2.083166e-11,2.083166e-11,1.330819,-1.974452,0.212555,-9.896050,-0.298452


In [104]:
nx.shortest_path_length( train_graph, source=1484, target=2784)

9

In [113]:
xtrain1

,node_1,node_2,link,jaccard_distance,cosine_distance,shortest_path_length,same_community,common_neighbors,adamic_adar,resource_allocation_index,...,svd_v_s_5,svd_v_s_6,svd_v_d_1,svd_v_d_2,svd_v_d_3,svd_v_d_4,svd_v_d_5,svd_v_d_6,svd_dot_u,svd_dot_v
0,3413,3343,1,0.294118,0.047619,1,1,5,2.161036,0.501354,...,2.643381e-04,3.930846e-05,3.168043e-04,-2.126518e-03,-9.219622e-04,-9.834833e-04,2.151501e-04,3.207119e-05,8.625683e-06,8.625683e-06
1,146,2234,1,0.000000,0.000000,1,1,0,0.000000,0.000000,...,1.629504e-07,5.308601e-07,6.328826e-08,-2.645776e-06,-9.527660e-07,-2.180966e-06,5.250280e-09,1.054386e-08,2.157490e-10,2.157490e-10
2,1481,1113,1,0.176471,0.031250,1,1,3,1.392175,0.348485,...,1.667227e-07,1.769006e-07,-1.101708e-04,-3.129206e-05,-1.735056e-04,-3.601277e-05,1.821458e-07,1.118372e-06,3.087866e-08,3.087866e-08
3,2036,2350,1,0.505618,0.010302,1,1,45,11.265735,0.839549,...,7.125375e-04,1.404221e-01,1.009403e-04,-2.740421e-03,-4.701884e-03,6.149393e-02,-7.842732e-04,1.308478e-01,1.262273e-02,1.262273e-02
4,65,2031,1,0.529412,0.053571,1,1,9,3.733518,0.817671,...,2.221650e-03,3.978645e-04,2.426358e-04,-1.585391e-03,-6.544714e-04,-7.730326e-04,1.511492e-04,2.750821e-05,2.179123e-05,2.179123e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27613,1787,3286,0,0.000000,0.000000,9,0,0,0.000000,0.000000,...,1.631376e-05,3.354205e-06,3.273391e-10,-6.982458e-09,-2.430715e-09,-1.004038e-09,3.579229e-12,4.689558e-13,9.583323e-12,9.583323e-12
27614,3373,2483,0,0.000000,0.000000,6,0,0,0.000000,0.000000,...,2.705416e-09,3.992914e-09,-6.830347e-04,1.073930e-02,1.426872e-02,-1.636402e-01,2.227113e-04,3.582600e-02,7.461572e-08,7.461572e-08
27615,1496,549,0,0.000000,0.000000,5,0,0,0.000000,0.000000,...,2.525372e-05,2.773948e-06,1.316703e-05,-3.045608e-05,-1.476499e-05,-2.530197e-05,1.098556e-06,3.076463e-07,4.725180e-08,4.725180e-08
27616,1484,2784,0,0.000000,0.000000,9,0,0,0.000000,0.000000,...,3.446323e-07,8.399597e-09,6.207574e-07,-3.054699e-06,-1.083077e-06,-9.646349e-07,6.801768e-08,3.836097e-09,2.083166e-11,2.083166e-11


In [109]:
Base=[]
Base += ['jaccard_distance', 'cosine_distance', 'shortest_path', 'same_community', 
        'Common_neighbors', 'adar', 'Resource_Allocation_Index', 'preferential_attachment',
        'svd_dot_u', 'svd_dot_v']

In [110]:
df_final_train[['jaccard_distance', 'cosine_distance', 'shortest_path_length', 'same_community', 
        'common_neighbors', 'adamic_adar', 'resource_allocation_index', 'preferential_attachment',
        'svd_dot_u', 'svd_dot_v']]

KeyError: "['shortest_path_length', 'common_neighbors', 'adamic_adar', 'resource_allocation_index', 'preferential_attachment'] not in index"

In [ ]:
# np.where(np.isnan(df_final_train))
print(df_final_train[Base]==xtrain1[Base])
print(df_final_test[Base]==xtest1[Base])

In [ ]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform

# estimators = [10,50,100,250,450]
# train_scores = []
# test_scores = []
# for i in estimators:
#     clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=5, max_features='auto', max_leaf_nodes=None,
#             min_impurity_decrease=0.0,
#             min_samples_leaf=52, min_samples_split=120,
#             min_weight_fraction_leaf=0.0, n_estimators=i, n_jobs=-1,random_state=25,verbose=0,warm_start=False)
#     clf.fit(df_final_train,y_train)
#     train_sc = f1_score(y_train,clf.predict(df_final_train))
#     test_sc = f1_score(y_test,clf.predict(df_final_test))
#     test_scores.append(test_sc)
#     train_scores.append(train_sc)
#     print('Estimators = ',i,'Train Score',train_sc,'test Score',test_sc)
# plt.plot(estimators,train_scores,label='Train Score')
# plt.plot(estimators,test_scores,label='Test Score')
# plt.xlabel('Estimators')
# plt.ylabel('Score')
# plt.title('Estimators vs score at depth of 5')

In [ ]:
# depths = [3,9,11,15,20,35,50,70,130]
# train_scores = []
# test_scores = []
# for i in depths:
#     clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#             max_depth=i, max_features='auto', max_leaf_nodes=None,
#             min_impurity_decrease=0.0,
#             min_samples_leaf=52, min_samples_split=120,
#             min_weight_fraction_leaf=0.0, n_estimators=115, n_jobs=-1,random_state=25,verbose=0,warm_start=False)
#     clf.fit(df_final_train,y_train)
#     train_sc = f1_score(y_train,clf.predict(df_final_train))
#     test_sc = f1_score(y_test,clf.predict(df_final_test))
#     test_scores.append(test_sc)
#     train_scores.append(train_sc)
#     print('depth = ',i,'Train Score',train_sc,'test Score',test_sc)
# plt.plot(depths,train_scores,label='Train Score')
# plt.plot(depths,test_scores,label='Test Score')
# plt.xlabel('Depth')
# plt.ylabel('Score')
# plt.title('Depth vs score at depth of 5 at estimators = 115')
# plt.show()

In [ ]:
param_dist = {"n_estimators":sp_randint(105,125),
              "max_depth": sp_randint(10,15),
              "min_samples_split": sp_randint(110,190),
              "min_samples_leaf": sp_randint(25,65)}

clf = RandomForestClassifier(random_state=25,n_jobs=-1)

rf_random = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=5,cv=10,scoring='f1',random_state=25)


rf_random.fit(df_final_train,y_train)
print('mean test scores',rf_random.cv_results_['mean_test_score'])
# print('mean train scores',rf_random.cv_results_['mean_train_score'])

In [ ]:
print(rf_random.best_estimator_)

In [ ]:
clf = rf_random.best_estimator_

In [ ]:
clf.fit(df_final_train,y_train)
y_train_pred = clf.predict(df_final_train)
y_test_pred = clf.predict(df_final_test)

In [ ]:
from sklearn.metrics import f1_score
print('Train f1 score',f1_score(y_train,y_train_pred))
print('Test f1 score',f1_score(y_test,y_test_pred))

In [ ]:
# f = open("./results/RandomForest_f1_transposed.csv", "a")
# f.write(str(f1_score(y_test,y_test_pred)))
# if not args.PCA:
#     f.write('\n')
# else:
#     f.write(',')
# f.close()

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(test_y, predict_y,split):
    C = confusion_matrix(test_y, predict_y)
    
    A =(((C.T)/(C.sum(axis=1))).T)
    
    B =(C/C.sum(axis=0))
    plt.figure(figsize=(20,4))
    
    labels = [0,1]
    # representing A in heatmap format
    cmap=sns.light_palette("blue")
    plt.subplot(1, 3, 1)
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Confusion matrix")
    
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")
    
    plt.subplot(1, 3, 3)
    # representing B in heatmap format
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")
    plt.tight_layout()
#     plt.savefig(f"./results/{PCAornot}/{genre}/RandomForest/{split}_confusion_matrix.png")
    
    plt.show()

In [ ]:
print('Train confusion_matrix')
plot_confusion_matrix(y_train,y_train_pred,'Train')
print('Test confusion_matrix')
plot_confusion_matrix(y_test,y_test_pred,'Test')

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr,tpr,ths = roc_curve(y_test,y_test_pred)
auc_sc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='navy',label='ROC curve (area = %0.2f)' % auc_sc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic with test eiko')
plt.legend()
plt.tight_layout()
# plt.savefig(f"./results/{PCAornot}/{genre}/RandomForest/roc_curve.png")
plt.show()

In [ ]:
features = df_final_train.columns
importances = clf.feature_importances_
indices = (np.argsort(importances))[-25:]
plt.figure(figsize=(10,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='r', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.tight_layout()
# plt.savefig(f"./results/{PCAornot}/{genre}/RandomForest/importance.png")
plt.show()

# GBDT

In [ ]:
#reading
from pandas import read_hdf
if args.PCA:
    df_final_train = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage4.h5', 'train_df',mode='r')
    df_final_test = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage4.h5', 'test_df',mode='r')
else:
    df_final_train = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage3.h5', 'train_df',mode='r')
    df_final_test = read_hdf(f'./eiko/{genre}/fea_sample/other_features/storage_sample_stage3.h5', 'test_df',mode='r')  

In [ ]:
df_final_train

In [ ]:
df_final_train.columns

In [ ]:
y_train = df_final_train.indicator_link
y_test = df_final_test.indicator_link

In [ ]:
df_final_train.drop(['source_node', 'destination_node','indicator_link','svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6','svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6','svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6','svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6','shortest_path'],axis=1,inplace=True)
df_final_test.drop(['source_node', 'destination_node','indicator_link','svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6','svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6','svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6','svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6','shortest_path'],axis=1,inplace=True)

# df_final_train=df_final_train[['jaccard_distance','cosine_distance','same_community','Common_neighbors']]
# df_final_test=df_final_test[['jaccard_distance','cosine_distance','same_community','Common_neighbors']]


In [ ]:
# df_final_train=df_final_train[['adar']]
# df_final_test=df_final_test[['adar']]

In [ ]:
# df_final_train=df_final_train[['Resource_Allocation_Index']]
# df_final_test=df_final_test[['Resource_Allocation_Index']]

In [ ]:
# df_final_train=df_final_train[['Preferential_Attachment']]
# df_final_test=df_final_test[['Preferential_Attachment']]

In [ ]:
# df_final_train=df_final_train[['node2vector']]
# df_final_test=df_final_test[['node2vector']]

In [ ]:
# df_final_train=df_final_train[['GEMSEC']]
# df_final_test=df_final_test[['GEMSEC']]

In [ ]:
# df_final_train=df_final_train[['GEMSECWithRegularization']]
# df_final_test=df_final_test[['GEMSECWithRegularization']]

In [ ]:
# df_final_train=df_final_train[['DeepWalk']]
# df_final_test=df_final_test[['DeepWalk']]

In [ ]:
# df_final_train=df_final_train[['DeepWalkWithRegularization']]
# df_final_test=df_final_test[['DeepWalkWithRegularization']]

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score

In [ ]:
param_dist = {"n_estimators":[10,50,100,250],
              "max_depth": [3,9,11,15,20,35,50,70,130]}



clf = GradientBoostingClassifier(random_state=25,verbose=1)

rf_random = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=5,cv=3,scoring='f1',random_state=25)

rf_random.fit(df_final_train,y_train)
print('mean test scores',rf_random.cv_results_['mean_test_score'])

In [ ]:
print(rf_random.best_estimator_)

In [ ]:
clf = rf_random.best_estimator_

In [ ]:
# clf = GradientBoostingClassifier(n_estimators=10,max_depth=2,random_state=25,verbose=1)

In [ ]:
clf.fit(df_final_train,y_train)
y_train_pred = clf.predict(df_final_train)
y_test_pred = clf.predict(df_final_test)

In [ ]:
from sklearn.metrics import f1_score
print('Train f1 score',f1_score(y_train,y_train_pred))
print('Test f1 score',f1_score(y_test,y_test_pred))
# cnt=0
# for i in y_test_pred:
#     if i==0:
#         cnt+=1
# print(cnt)        
# print(y_test)

In [ ]:
# f = open("./results/GBDT_f1_transposed.csv", "a")
# f.write(str(f1_score(y_test,y_test_pred)))
# if not args.PCA:
#     f.write('\n')
# else:
#     f.write(',')
# f.close()

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(test_y, predict_y, split):
    C = confusion_matrix(test_y, predict_y)
    
    A =(((C.T)/(C.sum(axis=1))).T)
    
    B =(C/C.sum(axis=0))
    plt.figure(figsize=(20,4))
    
    labels = [0,1]
    # representing A in heatmap format
    sns.set()
    cmap=sns.light_palette("blue")
    plt.subplot(1, 3, 1)
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Confusion matrix")
    
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")
    
    plt.subplot(1, 3, 3)
    # representing B in heatmap format
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")
    plt.tight_layout()
#     plt.savefig(f"./results/{PCAornot}/{genre}/GBDT/{split}_confusion_matrix.png")
    plt.show()

In [ ]:
print('Train confusion_matrix')
plot_confusion_matrix(y_train,y_train_pred,'Train')
print('Test confusion_matrix')
plot_confusion_matrix(y_test,y_test_pred,'Test')

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr,tpr,ths = roc_curve(y_test,y_test_pred)
auc_sc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='navy',label='ROC curve (area = %0.2f)' % auc_sc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic with test eiko')
plt.legend()
plt.tight_layout()
# plt.savefig(f"./results/{PCAornot}/{genre}/GBDT/roc_curve.png")
plt.show()

In [ ]:
features = df_final_train.columns
importances = clf.feature_importances_
indices = (np.argsort(importances))[-25:]
plt.figure(figsize=(10,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='r', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.tight_layout()
# plt.savefig(f"./results/{PCAornot}/{genre}/GBDT/importance.png")
plt.show()